In [4]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Classe Sistema_pmm3

In [5]:
class sistema_pmma3():
    """ Representa um sistema com três graus de liberdade e amortecimento proporcional"""

    def __init__(self,m1,m2,m3,k1,k2,k3,beta):
        self.m1= m1
        self.m2= m2
        self.m3= m3
        self.k1= k1
        self.k2= k2
        self.k3= k3
        self.a1= beta*k1
        self.a2= beta*k2
        self.a3= beta*k3
        self.w0= 0;
        self.w1= 0;
        self.w2= 0;
        self.Modes= []
        self.GenMass= []
        self.GenStiff= []
        
    def show(self):
        print("Massa 1: " + str(self.m1) + " Massa 2: " + str(self.m2) + " Massa 3: " + str(self.m3) )
        print("Rigidez 1: " + str(self.k1) + " Rigidez 2: " + str(self.k2) + " Rigidez 3: " + str(self.k3) )
        print("Amortecimento : " + str(self.a1) + " Amortecimento 2: " + \
              str(self.a2) + " Amortecimento 3: " + str(self.a3) )
    
    def ressonances(self,interval):
        
        m123  = self.m1*self.m2*self.m3
        k123  = self.k1*self.k2*self.k3
        m1_2_3= self.m1+self.m2+self.m3
        
        m2_3= self.m2+self.m3
        k2_3= self.k1+self.k3
        k1_2= self.k1+self.k2
        
        m13= self.m1*self.m2
        m23= self.m2*self.m3
        m12= self.m1*self.m2
        k23= self.k2*self.k3
        k13= self.k1*self.k3
        k12= self.k1*self.k2
        
        f = lambda w: m123*w**6 -(m13*k2_3+m23*k1_2+m12*self.k3)*w**4+(m1_2_3*k23+m2_3*k13+self.m3*k12)*w**2-k123 
        domain= np.arange(0,interval)
        roots= []
        
        for bogey in domain:
            new_root= round(abs(fsolve(f,bogey)[0]),2)
            if new_root not in roots:
                roots.append(new_root)
            
        return np.sort(roots)
    
    def modes(self,omega):
        X1= 1
        X3= self.k2/( (self.k2+self.k3-self.m2*omega**2)*(1-(self.m3/self.k3)*omega**2)-self.k3)
        X2= X3*(1-(self.m3/self.k3)*omega**2)
        
        return np.array([[X1],[X2],[X3]])
    
    def setFrequencies(self,omegas):    
        self.w0= omegas[0]
        self.w1= omegas[1]
        self.w2= omegas[2]
                
    def getModes(self):
        shape_0 = sistema.modes(omegas[0])
        shape_1 = sistema.modes(omegas[1])
        shape_2 = sistema.modes(omegas[2])
        self.Modes= np.concatenate((shape_0,shape_1,shape_2),axis=1)
        return self.Modes
    
    def getGenMass(self):
        Mass= np.diag([self.m1,self.m2,self.m3])
        return np.matmul(np.matmul(np.transpose(Modes),Mass),Modes) 
            
    def getGenStiff(self):
        Stiff= [[self.k1+self.k2,-self.k2,0],
                [-self.k2,self.k2+self.k3,-self.k3],
                [0,-self.k3,self.k3]]
        return np.matmul(np.matmul(np.transpose(Modes),Stiff),Modes) 
    
    def getGenDamp(self):
        Damp= [[self.a1+self.a2,-self.a2,0],
                [-self.a2,self.a2+self.a3,-self.a3],
                [0,-self.a3,self.a3]]
        return np.matmul(np.matmul(np.transpose(Modes),Damp),Modes) 
        
    def getResponse(self,omega):
        pass
                

In [7]:
sistema= sistema_pmma3(79.2,79.2,79.2,1000000,1000000,1000000,0.0005)
sistema.show()

Massa 1: 79.2 Massa 2: 79.2 Massa 3: 79.2
Rigidez 1: 1000000 Rigidez 2: 1000000 Rigidez 3: 1000000
Amortecimento : 500.0 Amortecimento 2: 500.0 Amortecimento 3: 500.0


In [7]:
omegas= sistema.ressonances(10)
print(omegas)

[ 50.01 140.12 202.48]


In [8]:
sistema.setFrequencies(omegas)
Modes= sistema.getModes()
print(Modes)

[[ 1.          1.          1.        ]
 [ 1.80207831  0.44506216 -1.24684193]
 [ 2.24720238 -0.80193944  0.55487861]]


In [6]:
DiagMass= sistema.getGenMass()
DiagMass

array([[ 7.36354458e+02, -6.60116751e-03,  1.40420712e-03],
       [-6.60116751e-03,  1.45822026e+02,  7.82386424e-03],
       [ 1.40420712e-03,  7.82386424e-03,  2.26710402e+02]])

In [7]:
DiagStiff= sistema.getGenStiff()
DiagStiff

array([[ 1.84146505e+06, -1.74029414e+02, -1.54007971e+02],
       [-1.74029414e+02,  2.86296901e+06,  1.09186097e+02],
       [-1.54007971e+02,  1.09186097e+02,  9.29449560e+06]])

# Verificação

In [8]:
frequencies= np.divide(DiagStiff,DiagMass)
w0_estimado= np.sqrt(frequencies[0][0])
w1_estimado= np.sqrt(frequencies[1][1]) 
w2_estimado= np.sqrt(frequencies[2][2])
print('w0 é: ' + str(w0_estimado))
print('w1 é: ' + str(w1_estimado))
print('w2 é: ' + str(w2_estimado))

w0 é: 50.00786119630546
w1 é: 140.11891289293868
w2 é: 202.4776954341178


# Respostas em Frequencia

In [9]:
teste_modes= np.matmul(np.transpose(Modes),Modes)
print('Testando a qualidade da matrix modal: ')
print(teste_modes)

Testando a qualidade da matrix modal: 
[[ 9.29740478e+00 -8.33480746e-05  1.77298879e-05]
 [-8.33480746e-05  1.84118720e+00  9.87861647e-05]
 [ 1.77298879e-05  9.87861647e-05  2.86250508e+00]]


# Fatores de Amortecimento

In [10]:
DiagDamp= sistema.getGenDamp()
DiagDamp

array([[ 9.20732527e+02, -8.70147070e-02, -7.70039857e-02],
       [-8.70147070e-02,  1.43148450e+03,  5.45930485e-02],
       [-7.70039857e-02,  5.45930485e-02,  4.64724780e+03]])